# 1: Rating Estimation

<img src="./table.jpg" width="450px" height="200px" title="problem"/>

In [16]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

user1 = np.array([1,0,1,0,0,-2])
user2 = np.array([2,-1,0,0,0,-1])
user3 = np.array([4/3, 0, 0, 1/3, -5/3, 0])
user4 = np.array([-7/4, 5/4, -3/4, 0, 5/4, 0])
user5 = np.array([-1, 0, 1, -1, 1, 0])

print('Pearson correlation coefficient를 기반으로 user1과의 유사도를 계산하면 이하와 같다.')

print('user2 :','%.3f' %cos_sim(user1, user2))
print('user3 :','%.3f' %cos_sim(user1, user3))
print('user4 :','%.3f' %cos_sim(user1, user4))
print('user5 :','%.3f' %cos_sim(user1, user5))

Pearson correlation coefficient를 기반으로 user1과의 유사도를 계산하면 이하와 같다.
user2 : 0.667
user3 : 0.252
user4 : -0.393
user5 : 0.000


- 위 유사도 계산 결과를 바탕으로, user1과 가장 유사한 사용자는 user2와 user3이라고 볼 수 있다.
- 특히 user2와의 유사도가 0.667로 가장 높게 나타났으며 user3은 0.252로 유사도가 상대적으로 낮다.
- 영화 rating이 5점 척도라고 가정할 때 user2가 item4에 2점을 부여하였고 user3이 3점을 부여하였으므로, user1의 item4에 대한 rating은 2점일 것이라고 추정할 수 있다. 

# 2: Recommendation system
- 사용자 아이디 10번의 추천영화는?
- 잠재 노드의 개수를 늘이거나 줄이면 어떠한 영향이 있는가?

In [17]:
import pandas as pd

movies = pd.read_csv("C:/Users/min/Documents/SocialCom/Recommendation/ml-latest-small/movies.csv")
ratings = pd.read_csv("C:/Users/min/Documents/SocialCom/Recommendation/ml-latest-small/ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

rating_movies = pd.merge(ratings, movies, on='movieId')
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [18]:
ratings_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [19]:
from sklearn.metrics import mean_squared_error
 
def get_rmse(R, P, Q, non_zeros):
    error = 0
    ''' Create the predicted R matrix by multiplying the decomposed matrices P and Q.T'''
    full_pred_matrix = np.dot(P, Q.T)
     
    ''' Extract the indices of non-zero values from the actual R matrix 
         and calculate the RMSE btw the actual and predicted values '''
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
     
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
       
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
     
    return rmse


def matrix_factorization(R, K, steps, learning_rate, r_lambda):
    num_users, num_items = R.shape
    ''' Initialize matrices P and Q with random values from a normal distribution, specifying their size '''
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
 
    break_count = 0
        
    '''Store the positions (row, column, value) where R > 0 in the non_zeros list'''
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
     
    '''Update matrices P and Q using Stochastic Gradient Descent (SGD) technique'''
    for step in range(steps):
        for i, j, r in non_zeros:
            
            '''Compute the difference btw actual and predicted values'''
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
         
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
             
    return P, Q

In [20]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)

### iteration step :  0  rmse :  2.9023619751336867
### iteration step :  10  rmse :  0.7335768591017927
### iteration step :  20  rmse :  0.5115539026853442
### iteration step :  30  rmse :  0.37261628282537446
### iteration step :  40  rmse :  0.2960818299181014
### iteration step :  50  rmse :  0.2520353192341642
### iteration step :  60  rmse :  0.22487503275269854
### iteration step :  70  rmse :  0.20685455302331537
### iteration step :  80  rmse :  0.19413418783028685
### iteration step :  90  rmse :  0.18470082002720403
### iteration step :  100  rmse :  0.17742927527209104
### iteration step :  110  rmse :  0.17165226964707486
### iteration step :  120  rmse :  0.1669518194687172
### iteration step :  130  rmse :  0.16305292191997542
### iteration step :  140  rmse :  0.15976691929679643
### iteration step :  150  rmse :  0.1569598699945732
### iteration step :  160  rmse :  0.15453398186715428
### iteration step :  170  rmse :  0.15241618551077643
### iteration step :  180  r

In [21]:
pred_matrix = np.dot(P, Q.T)

In [22]:
ratings_pred_matrix = pd.DataFrame(data = pred_matrix, index=ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [24]:
userId = 10

user_rating = ratings_matrix.loc[userId, :]
already_seen = user_rating[user_rating>0].index.tolist()
movies_list = ratings_matrix.columns.tolist()
unseen_list = [movie for movie in movies_list if movie not in already_seen]

ratings_pred_matrix.loc[userId, unseen_list].sort_values(ascending=False)[:5]

title
It's a Wonderful Life (1946)         5.549736
Thank You for Smoking (2006)         5.388436
Postman, The (Postino, Il) (1994)    5.300148
First Knight (1995)                  5.270648
Little Mermaid, The (1989)           5.196844
Name: 10, dtype: float64

user10에게 5개의 영화를 추천한다면 이하의 영화가 제시될 것이다.
1. It's a Wonderful Life (1946)
2. Thank You for Smoking (2006)
3. Postman, The (Postino, Il) (1994)
4. First Knight (1995)
5. Little Mermaid, The (1989)
6. Little Mermaid, The (1989)

한편, 잠재 노드의 개수는 하이퍼파라미터 K값으로 조절될 수 있는데, 이번에는 K값이 추천 결과에 어떠한 영향을 주는지 알아보도록 할 것이다.

In [26]:
P2, Q2 = matrix_factorization(ratings_matrix.values, K=20, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix2 = np.dot(P2, Q2.T)
ratings_pred_matrix2 = pd.DataFrame(data = pred_matrix2, index=ratings_matrix.index, columns = ratings_matrix.columns)

### iteration step :  0  rmse :  2.7363177147457125
### iteration step :  10  rmse :  0.7279847466900063
### iteration step :  20  rmse :  0.5668646954963797
### iteration step :  30  rmse :  0.4923255614623767
### iteration step :  40  rmse :  0.45293307060211574
### iteration step :  50  rmse :  0.42914791260195284
### iteration step :  60  rmse :  0.41351127815895716
### iteration step :  70  rmse :  0.40250962352488245
### iteration step :  80  rmse :  0.3943478593818473
### iteration step :  90  rmse :  0.3880404530079418
### iteration step :  100  rmse :  0.3830072658343339
### iteration step :  110  rmse :  0.3788864602854821
### iteration step :  120  rmse :  0.37544154087650733
### iteration step :  130  rmse :  0.37251183965220963
### iteration step :  140  rmse :  0.369984556126001
### iteration step :  150  rmse :  0.36777819465997263
### iteration step :  160  rmse :  0.3658323725936413
### iteration step :  170  rmse :  0.36410134551223655
### iteration step :  180  rmse 

In [27]:
P3, Q3 = matrix_factorization(ratings_matrix.values, K=80, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix3 = np.dot(P3, Q3.T)
ratings_pred_matrix3 = pd.DataFrame(data = pred_matrix3, index=ratings_matrix.index, columns = ratings_matrix.columns)

### iteration step :  0  rmse :  2.972826738843401
### iteration step :  10  rmse :  0.7434136611044098
### iteration step :  20  rmse :  0.5135336301350819
### iteration step :  30  rmse :  0.3481507112940733
### iteration step :  40  rmse :  0.2511022209604655
### iteration step :  50  rmse :  0.1954125247492782
### iteration step :  60  rmse :  0.16245889373230768
### iteration step :  70  rmse :  0.14193277747125768
### iteration step :  80  rmse :  0.12840470224364903
### iteration step :  90  rmse :  0.11901821575016575
### iteration step :  100  rmse :  0.11221984841467007
### iteration step :  110  rmse :  0.10712228854537909
### iteration step :  120  rmse :  0.10319166030624852
### iteration step :  130  rmse :  0.10009090449307714
### iteration step :  140  rmse :  0.09759815185004017
### iteration step :  150  rmse :  0.09556208360724176
### iteration step :  160  rmse :  0.09387633132632078
### iteration step :  170  rmse :  0.09246415844178534
### iteration step :  180  r

- k값이 min(행의 크기, 열의 크기)보다 크면 matrix factorization하여 쪼개진 matrix끼리 다시 곱했을 때 원본 값과 완전 일치하게 된다. 
- k값이 커질수록 rmse값이 점차 작아지나, 결과적으로 추천 모델의 성능이 점차 떨어진다는 문제가 발생한다.